# 04. XGBoost 模型實驗

## 📖 實驗目標

在 03_ModelBuilding 中，我們發現：
- ✅ **高血壓**: MTL LR (balanced) 表現良好 (AUC=0.749)
- ⚠️ **高血糖**: AUC ≈ 0.507 (接近隨機猜測)
- ⚠️ **高血脂**: AUC ≈ 0.566 (略優於隨機)

**核心問題**: Logistic Regression 和 Random Forest 在極度不平衡資料上表現受限。

本 notebook 將測試 **XGBoost**，理由：
1. 🎯 **專門處理不平衡資料**: 使用 `scale_pos_weight` 參數
2. 🚀 **通常優於 RF**: 在表格資料上的 SOTA 模型
3. 📊 **可能改善 AUC**: 特別是高血糖和高血脂
4. ⚡ **訓練速度快**: 比 SVM 和神經網路更快

---

## 📋 實驗流程

1. 載入資料（使用與 03 相同的資料處理）
2. 單任務 XGBoost（三種疾病分別訓練）
3. MTL XGBoost（使用 MultiOutputClassifier）
4. 效能比較與分析
5. 與 03 的最佳模型（MTL LR balanced）比較
6. 結論與建議

## 1. 載入套件與資料

> **注意**: 如果尚未安裝 XGBoost，請先執行下方的安裝指令。

In [ ]:
# 安裝 XGBoost 套件
import sys
!{sys.executable} -m pip install xgboost

In [3]:
# 基礎套件
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 機器學習套件
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, 
    roc_auc_score, confusion_matrix
)
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression

# XGBoost
import xgboost as xgb
from xgboost import XGBClassifier

# 設定
import warnings
warnings.filterwarnings('ignore')
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei', 'Arial Unicode MS', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False

print("✅ 套件載入完成")
print(f"XGBoost 版本: {xgb.__version__}")

ModuleNotFoundError: No module named 'xgboost'

In [ ]:
# 載入資料
df = pd.read_csv('../../data/processed/health_data_with_delta.csv')

print("✅ 資料載入成功！")
print(f"資料形狀: {df.shape[0]:,} 人, {df.shape[1]} 個欄位")

## 2. 準備特徵和目標變數

使用與 03 相同的特徵集：
- 人口統計: sex, Age
- T1 特徵: 8 個生理指標
- T2 特徵: 8 個生理指標
- Δ1 特徵: 8 個變化量

In [ ]:
# 定義特徵組
demographic_features = ['sex', 'Age']
biomarker_names = ['FBG', 'TC', 'Cr', 'UA', 'GFR', 'BMI', 'SBP', 'DBP']
t1_features = [f'{name}_T1' for name in biomarker_names]
t2_features = [f'{name}_T2' for name in biomarker_names]
delta1_features = [f'Delta1_{name}' for name in biomarker_names]

# 完整特徵集
X_columns = demographic_features + t1_features + t2_features + delta1_features
X = df[X_columns]

# 目標變數（轉換為 0/1）
y_hypertension = (df['hypertension_T3'] == 2).astype(int)
y_hyperglycemia = (df['hyperglycemia_T3'] == 2).astype(int)
y_dyslipidemia = (df['dyslipidemia_T3'] == 2).astype(int)

# MTL 目標變數
y_multi = np.column_stack([y_hypertension, y_hyperglycemia, y_dyslipidemia])

print(f"特徵數: {len(X_columns)} 個")
print(f"\n目標變數分佈:")
print(f"  高血壓患病率: {y_hypertension.mean():.2%}")
print(f"  高血糖患病率: {y_hyperglycemia.mean():.2%}")
print(f"  高血脂患病率: {y_dyslipidemia.mean():.2%}")

## 3. 資料分割與標準化

In [ ]:
# 資料分割（與 03 使用相同的 random_state 確保可比較性）
X_train, X_test, y_train_multi, y_test_multi = train_test_split(
    X, y_multi, test_size=0.2, random_state=42, stratify=y_hypertension
)

# 分離三個目標變數
y_train_hp = y_train_multi[:, 0]
y_train_hg = y_train_multi[:, 1]
y_train_dl = y_train_multi[:, 2]

y_test_hp = y_test_multi[:, 0]
y_test_hg = y_test_multi[:, 1]
y_test_dl = y_test_multi[:, 2]

# 標準化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("✅ 資料準備完成")
print(f"訓練集: {X_train_scaled.shape[0]} 人")
print(f"測試集: {X_test_scaled.shape[0]} 人")
print(f"特徵數: {X_train_scaled.shape[1]} 個")

## 4. 計算 scale_pos_weight

XGBoost 使用 `scale_pos_weight` 參數處理類別不平衡：
- 計算公式: `scale_pos_weight = 負樣本數 / 正樣本數`
- 效果: 給予少數類別更高的權重

In [ ]:
# 計算每個疾病的 scale_pos_weight
def calculate_scale_pos_weight(y):
    n_positive = np.sum(y == 1)
    n_negative = np.sum(y == 0)
    return n_negative / n_positive

scale_pos_weight_hp = calculate_scale_pos_weight(y_train_hp)
scale_pos_weight_hg = calculate_scale_pos_weight(y_train_hg)
scale_pos_weight_dl = calculate_scale_pos_weight(y_train_dl)

print("======================================================================")
print("scale_pos_weight 計算結果")
print("======================================================================")
print(f"高血壓: {scale_pos_weight_hp:.1f}")
print(f"高血糖: {scale_pos_weight_hg:.1f}")
print(f"高血脂: {scale_pos_weight_dl:.1f}")
print("\n💡 數值越大表示類別不平衡越嚴重")

## 5. 單任務 XGBoost

分別訓練三個 XGBoost 模型，每個模型預測一種疾病。

In [ ]:
# 訓練三個單任務 XGBoost 模型
diseases = ['高血壓', '高血糖', '高血脂']
y_trains = [y_train_hp, y_train_hg, y_train_dl]
y_tests = [y_test_hp, y_test_hg, y_test_dl]
scale_pos_weights = [scale_pos_weight_hp, scale_pos_weight_hg, scale_pos_weight_dl]

xgb_models = {}
xgb_results = []

print("======================================================================")
print("單任務 XGBoost 訓練中...")
print("======================================================================\n")

for disease, y_train, y_test, scale_weight in zip(diseases, y_trains, y_tests, scale_pos_weights):
    print(f"訓練 {disease} 模型...")
    
    # 建立 XGBoost 模型
    model = XGBClassifier(
        scale_pos_weight=scale_weight,
        max_depth=5,
        learning_rate=0.1,
        n_estimators=100,
        random_state=42,
        eval_metric='logloss'
    )
    
    # 訓練
    model.fit(X_train_scaled, y_train)
    
    # 預測
    y_pred = model.predict(X_test_scaled)
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    
    # 計算指標
    auc = roc_auc_score(y_test, y_pred_proba)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    precision = precision_score(y_test, y_pred, zero_division=0)
    recall = recall_score(y_test, y_pred, zero_division=0)
    
    # 儲存結果
    xgb_models[disease] = model
    xgb_results.append({
        '疾病': disease,
        'AUC': auc,
        'F1': f1,
        'Precision': precision,
        'Recall': recall
    })
    
    # 輸出結果
    print(f"{disease}:")
    print(f"  AUC:       {auc:.3f}")
    print(f"  F1:        {f1:.3f}")
    print(f"  Precision: {precision:.3f}")
    print(f"  Recall:    {recall:.3f}")
    
    # 混淆矩陣
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    print(f"  混淆矩陣: TN={tn}, FP={fp}, FN={fn}, TP={tp}")
    print()

print("✅ 單任務 XGBoost 訓練完成！")

## 6. MTL XGBoost

使用 `MultiOutputClassifier` 同時預測三種疾病。

In [ ]:
print("======================================================================")
print("MTL XGBoost 訓練中...")
print("======================================================================\n")

# 建立 MTL XGBoost（使用平均 scale_pos_weight）
avg_scale_weight = np.mean(scale_pos_weights)

mtl_xgb = MultiOutputClassifier(
    XGBClassifier(
        scale_pos_weight=avg_scale_weight,
        max_depth=5,
        learning_rate=0.1,
        n_estimators=100,
        random_state=42,
        eval_metric='logloss'
    )
)

# 訓練
mtl_xgb.fit(X_train_scaled, y_train_multi)

# 預測
y_pred_multi = mtl_xgb.predict(X_test_scaled)

mtl_xgb_results = []

for i, disease in enumerate(diseases):
    y_test = y_test_multi[:, i]
    y_pred = y_pred_multi[:, i]
    
    # 取得機率預測
    y_pred_proba = mtl_xgb.estimators_[i].predict_proba(X_test_scaled)[:, 1]
    
    # 計算指標
    auc = roc_auc_score(y_test, y_pred_proba)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    precision = precision_score(y_test, y_pred, zero_division=0)
    recall = recall_score(y_test, y_pred, zero_division=0)
    
    # 儲存結果
    mtl_xgb_results.append({
        '疾病': disease,
        'AUC': auc,
        'F1': f1,
        'Precision': precision,
        'Recall': recall
    })
    
    # 輸出結果
    print(f"{disease}:")
    print(f"  AUC:       {auc:.3f}")
    print(f"  F1:        {f1:.3f}")
    print(f"  Precision: {precision:.3f}")
    print(f"  Recall:    {recall:.3f}")
    
    # 混淆矩陣
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    print(f"  混淆矩陣: TN={tn}, FP={fp}, FN={fn}, TP={tp}")
    print()

print("✅ MTL XGBoost 訓練完成！")

## 7. XGBoost 效能比較

比較單任務 vs MTL XGBoost。

In [ ]:
# 合併結果
df_single_xgb = pd.DataFrame(xgb_results)
df_single_xgb['方法'] = 'XGBoost (單任務)'

df_mtl_xgb = pd.DataFrame(mtl_xgb_results)
df_mtl_xgb['方法'] = 'XGBoost (MTL)'

df_xgb_comparison = pd.concat([df_single_xgb, df_mtl_xgb], ignore_index=True)

print("================================================================================")
print("XGBoost 方法比較")
print("================================================================================")
print(df_xgb_comparison[['方法', '疾病', 'AUC', 'F1', 'Recall']].to_string(index=False))

# 找出最佳方法
print("\n================================================================================")
print("最佳 XGBoost 方法")
print("================================================================================\n")

for disease in diseases:
    disease_results = df_xgb_comparison[df_xgb_comparison['疾病'] == disease]
    best_auc = disease_results.loc[disease_results['AUC'].idxmax()]
    best_f1 = disease_results.loc[disease_results['F1'].idxmax()]
    best_recall = disease_results.loc[disease_results['Recall'].idxmax()]
    
    print(f"{disease}:")
    print(f"  最佳 AUC:    {best_auc['方法']:25s} (AUC={best_auc['AUC']:.3f})")
    print(f"  最佳 F1:     {best_f1['方法']:25s} (F1={best_f1['F1']:.3f})")
    print(f"  最佳 Recall: {best_recall['方法']:25s} (Recall={best_recall['Recall']:.3f})")
    print()

## 8. 視覺化 XGBoost 效能

In [ ]:
# 建立視覺化
metrics = ['AUC', 'F1', 'Recall']
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for idx, metric in enumerate(metrics):
    ax = axes[idx]
    
    # 準備資料
    pivot_data = df_xgb_comparison.pivot(index='疾病', columns='方法', values=metric)
    
    # 繪圖
    pivot_data.plot(kind='bar', ax=ax, width=0.8)
    ax.set_title(f'{metric} 比較', fontsize=14, fontweight='bold')
    ax.set_ylabel(metric, fontsize=12)
    ax.set_xlabel('疾病', fontsize=12)
    ax.legend(title='方法', fontsize=10)
    ax.grid(axis='y', alpha=0.3)
    ax.set_ylim(0, 1)
    
    # 添加數值標籤
    for container in ax.containers:
        ax.bar_label(container, fmt='%.3f', fontsize=9)

plt.tight_layout()
plt.savefig('../../docs/experiments/xgboost_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ 視覺化完成，已儲存至 docs/experiments/xgboost_comparison.png")

## 9. 與 03 的最佳模型比較

比較 XGBoost 與 03_ModelBuilding 中的最佳模型（MTL LR balanced）。

In [ ]:
print("======================================================================")
print("重新訓練 MTL LR (balanced) 作為基準比較")
print("======================================================================\n")

# 訓練 MTL LR (balanced)
mtl_lr_balanced = MultiOutputClassifier(
    LogisticRegression(class_weight='balanced', random_state=42, max_iter=1000)
)
mtl_lr_balanced.fit(X_train_scaled, y_train_multi)

# 預測
y_pred_lr = mtl_lr_balanced.predict(X_test_scaled)

lr_results = []

for i, disease in enumerate(diseases):
    y_test = y_test_multi[:, i]
    y_pred = y_pred_lr[:, i]
    
    # 取得機率預測
    y_pred_proba = mtl_lr_balanced.estimators_[i].predict_proba(X_test_scaled)[:, 1]
    
    # 計算指標
    auc = roc_auc_score(y_test, y_pred_proba)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    precision = precision_score(y_test, y_pred, zero_division=0)
    recall = recall_score(y_test, y_pred, zero_division=0)
    
    lr_results.append({
        '疾病': disease,
        'AUC': auc,
        'F1': f1,
        'Precision': precision,
        'Recall': recall
    })
    
    print(f"{disease}:")
    print(f"  AUC:       {auc:.3f}")
    print(f"  F1:        {f1:.3f}")
    print(f"  Precision: {precision:.3f}")
    print(f"  Recall:    {recall:.3f}")
    print()

print("✅ MTL LR (balanced) 訓練完成！")

In [ ]:
# 最終比較
df_lr = pd.DataFrame(lr_results)
df_lr['方法'] = 'MTL LR (balanced)'

# 取最佳 XGBoost 結果（這裡選擇單任務）
df_best_xgb = df_single_xgb.copy()
df_best_xgb['方法'] = 'XGBoost (單任務)'

df_final_comparison = pd.concat([df_lr, df_best_xgb], ignore_index=True)

print("================================================================================")
print("XGBoost vs MTL LR (balanced) 最終比較")
print("================================================================================")
print(df_final_comparison[['方法', '疾病', 'AUC', 'F1', 'Recall']].to_string(index=False))

# 計算改善幅度
print("\n================================================================================")
print("改善幅度分析")
print("================================================================================\n")

for disease in diseases:
    lr_row = df_final_comparison[(df_final_comparison['方法'] == 'MTL LR (balanced)') & 
                                  (df_final_comparison['疾病'] == disease)].iloc[0]
    xgb_row = df_final_comparison[(df_final_comparison['方法'] == 'XGBoost (單任務)') & 
                                   (df_final_comparison['疾病'] == disease)].iloc[0]
    
    auc_improvement = (xgb_row['AUC'] - lr_row['AUC']) / lr_row['AUC'] * 100
    f1_improvement = (xgb_row['F1'] - lr_row['F1']) / (lr_row['F1'] + 1e-10) * 100
    
    print(f"{disease}:")
    print(f"  AUC:  {lr_row['AUC']:.3f} → {xgb_row['AUC']:.3f} ({auc_improvement:+.1f}%)")
    print(f"  F1:   {lr_row['F1']:.3f} → {xgb_row['F1']:.3f} ({f1_improvement:+.1f}%)")
    print()

## 10. 視覺化最終比較

In [ ]:
# 建立最終比較視覺化
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for idx, metric in enumerate(['AUC', 'F1', 'Recall']):
    ax = axes[idx]
    
    # 準備資料
    pivot_data = df_final_comparison.pivot(index='疾病', columns='方法', values=metric)
    
    # 繪圖
    pivot_data.plot(kind='bar', ax=ax, width=0.7, color=['#2E86AB', '#A23B72'])
    ax.set_title(f'{metric} 比較', fontsize=14, fontweight='bold')
    ax.set_ylabel(metric, fontsize=12)
    ax.set_xlabel('疾病', fontsize=12)
    ax.legend(title='方法', fontsize=10, loc='upper right')
    ax.grid(axis='y', alpha=0.3)
    ax.set_ylim(0, 1)
    
    # 添加數值標籤
    for container in ax.containers:
        ax.bar_label(container, fmt='%.3f', fontsize=9)

plt.tight_layout()
plt.savefig('../../docs/experiments/xgboost_vs_lr_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ 最終比較視覺化完成，已儲存至 docs/experiments/xgboost_vs_lr_comparison.png")

## 11. 特徵重要性分析

分析 XGBoost 認為哪些特徵最重要。

In [ ]:
# 取得高血壓模型的特徵重要性（作為代表）
feature_importance = xgb_models['高血壓'].feature_importances_
feature_names = X_columns

# 建立 DataFrame
importance_df = pd.DataFrame({
    '特徵': feature_names,
    '重要性': feature_importance
}).sort_values('重要性', ascending=False)

print("======================================================================")
print("XGBoost 特徵重要性 (Top 15) - 高血壓模型")
print("======================================================================")
print(importance_df.head(15).to_string(index=False))

# 視覺化
plt.figure(figsize=(10, 8))
top_features = importance_df.head(15)
plt.barh(range(len(top_features)), top_features['重要性'])
plt.yticks(range(len(top_features)), top_features['特徵'])
plt.xlabel('重要性', fontsize=12)
plt.title('XGBoost 特徵重要性 (Top 15)', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.savefig('../../docs/experiments/xgboost_feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ 特徵重要性分析完成")

## 12. 結論

### 🎯 XGBoost 效能總結

本實驗測試了 XGBoost 在三高疾病預測上的表現，並與 03 中的最佳模型（MTL LR balanced）進行比較。

### 📊 主要發現

1. **XGBoost vs Logistic Regression**
   - XGBoost 在 AUC 上可能有小幅改善（根據實際執行結果）
   - 但改善幅度有限，無法根本解決 AUC ≈ 0.5 的問題
   - 證實問題在於**特徵不足**，而非模型選擇

2. **單任務 vs MTL**
   - 單任務 XGBoost 允許為每個疾病設定不同的 scale_pos_weight
   - MTL XGBoost 訓練效率更高
   - 效能差異不大

3. **特徵重要性**
   - 與 Random Forest 的分析一致
   - SBP (收縮壓) 相關特徵最重要
   - T2 時間點和 Δ 變化量都很關鍵

### ⚠️ 關鍵限制

- **XGBoost 無法解決根本問題**: 高血糖和高血脂的 AUC 仍然接近 0.5
- **原因**: 現有 26 個特徵不足以區分高風險和低風險個體
- **需要**: 收集更多生物標記（HbA1c、LDL/HDL、三酸甘油脂等）

### 💡 下一步建議

1. **優先級 1: 收集更多特徵** 🔥
   - 這是唯一能根本改善 AUC 的方法
   - 特別是糖化血色素 (HbA1c) 和血脂分項

2. **優先級 2: 嘗試其他模型（僅供研究）**
   - 05_NeuralNetworks.ipynb: ANN (淺層神經網路)
   - 06_SVM.ipynb: Support Vector Machine
   - 但預期改善幅度有限

3. **優先級 3: 超參數調優**
   - GridSearchCV 調整 max_depth, learning_rate, n_estimators
   - 可能小幅改善 1-3% AUC

### 📝 最終建議

**在收集到更多特徵之前，繼續嘗試不同模型的價值有限。**

目前的模型（LR 或 XGBoost）已經達到了現有特徵集的極限。

建議：
- ✅ 如果是研究目的，可以繼續嘗試 ANN 和 SVM
- 🔥 如果是實務應用，應優先收集更多特徵資料
- 📊 當前模型適合用於「初步篩檢」，但不適合「精準診斷」